# Project : Data Visualization 


## Questions: 
 
 ### 1. What is the Total number of Confirmed Cases? 
 ### 2. What is the Total number of Deaths Cases? 
 ### 3. What is the Total number of Recovered Cases? 
 ### 4. What is the Total number of (Confirmed , Deaths , Recovered) Cases of specific Date and Display map?
 ### 5. What is the Total number of (Confirmed , Deaths , Recovered) Cases of specific Country?
 ### 6. What is the Total number of (Confirmed , Deaths , Recovered) Cases of specific Continent?
####          6.1 Using Bar Chart 
####          6.2 Using Scatter Chart
####          6.3 Using Pie Chart 

In [9]:
import pandas as pd
import numpy as np
from datetime import date


import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go


import dash
import jupyter_dash
from dash import html, dcc, dash_table
from dash.html.H1 import H1
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

import pycountry_convert as pc

In [10]:
df = pd.read_csv("COVID-19-time-series-clean-complete.csv", index_col=0)


# total cases worldwide
aw1 = df.groupby(by='Date').sum()
last_day = aw1.iloc[-1]
total_cases = {'Confirmed' : last_day['Confirmed'],
              'Deaths' : last_day['Deaths'],
              'Recovered' : last_day['Active']}



# {df["Country/Region"].unique()}
STATE_LABELS = [{'label':str(continent), "value":str(continent)} for continent in df["Country/Region"].unique()]

external_stylesheets = [
    # Bootswatch theme
    dbc.themes.SLATE,
    # for social media icons
    "https://use.fontawesome.com/releases/v5.9.0/css/all.css",

]


STATE_LABELS.append({'label': 'ALL', 'value': 'ALL'})

In [11]:
# all cases from the begining till now
def get_stats(country):
    res = df[df["Country/Region"] == country].iloc[-1]
  
    stats={'Confirmed' : res['Confirmed'],
           'Deaths' : res['Deaths'],
           'Recovered': res['Active']}
    return stats

In [12]:
df_grouped_date = df.groupby(by=['Date', "Country/Region"]).sum()
def get_stats_by_date(country, date):
    df_date = df_grouped_date.loc[date].reset_index(level=0)
    res = df_date[df_date['Country/Region'] == country].iloc[0]
    stats={'Confirmed' : res['Confirmed'],
           'Deaths' : res['Deaths'],
           'Recovered': res['Active']}
    return stats

In [13]:
df_copy = df.copy()


In [14]:
def country_continent(column):
    Continent=[]
    for country in column:
        try:
            country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
            continent_code = pc.country_alpha2_to_continent_code(country_code)
            country_continent_name = pc.convert_continent_code_to_continent_name(continent_code)
            Continent.append(country_continent_name)
        except:
            Continent.append('None')
    return Continent


df_copy['Continent']=country_continent(df['Country/Region'])
# df.head()

# df_1 = df.groupby('Continent').sum().reset_index()

df_copy.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Continent
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0,Asia
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0,Europe
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0,Africa
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0,Europe
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0,Africa


In [15]:
df_1 = df_copy.groupby('Continent').sum().reset_index()

df_1["Continent"]= df_1["Continent"].replace("None", "Australia")
# df_1["Continent"]
df_1

,Continent,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,Africa,3683182737,90299103,1.055738e+09,2.537146e+09,11434503,250231,0
1,Asia,33708380827,467449291,9.284538e+09,2.395639e+10,129192648,1376049,-30
2,Europe,36767614726,644664457,5.406826e+09,3.071612e+10,190132389,1804293,0
3,Australia,24615286049,385198009,6.592137e+08,2.357087e+10,99364018,1044288,0
4,North America,4164450195,176468068,8.497219e+08,3.138260e+09,14694200,435253,0
5,Oceania,453520025,1464937,1.165049e+07,4.404046e+08,6668672,9249,0
6,South America,18064976126,527332622,5.940777e+09,1.159687e+10,56591429,1292585,0


In [16]:
# df_copy['New recovered'] = abs(df_copy['New recovered'])
total_confirmed = last_day['Confirmed']
total_deaths = last_day['Deaths']
total_recovered = last_day['Active']


In [17]:
ww = df_grouped_date.loc['2021-04-21'].reset_index(level=0)
df_top10 = ww.sort_values(by=['Confirmed'], ascending=[False]).head(10)
df_top10 = df_top10[['Country/Region', 'Confirmed', 'Deaths', 'Recovered']]

In [18]:
font_size = ".9vw"
color_active = "#F4F4F4"
color_inactive = "#AEAEAE"
color_bg = "#010914"

font_size_body = ".9vw"

In [19]:
# User row and col to control vertical alignment of logo/brand
navbar = [
    dbc.Row(
        [
            
            dbc.Col(
                html.A(
                    dbc.NavbarBrand(
                        [
                            html.P("COVID-19", className="navbar-brand-covid-19-text"),
                            html.P("Tracker", className="navbar-brand-us-cases-text"),
                        ]
                    ),
                    className="page-title-link",
                    href="/",
                )
            ),
        ],
        align="center",
        
    ),
    dbc.NavbarToggler(id="navbar-toggler", className="navbar-toggler-1"),
]


############################################
# cases Cards
############################################
confirmed_card = dbc.Card(
                        dbc.CardBody(
                                    [html.Br(),
                                    html.H1("Card title", className=f"top-bar-value-tested", id='confirmed-card'),
                                    html.P(f"Confirmed", className="card-text", style={'font-size':'18px'}), ]
                                ),
                        className=f"top-bar-card-tested",
                    )

############################################
deaths_card = dbc.Card(
                    dbc.CardBody([
                                 html.Br(),
                                 html.H1(className=f"top-bar-value-death rate", id='deaths-card'),
                                 html.P(f"Deaths", className="card-text", style={'font-size':'18px'}),]
                             ),
                    className=f"top-bar-card-death rate",
                )

############################################
recovered_card = dbc.Card(
            dbc.CardBody(
                [
                    html.Br(),
                    html.H1(className=f"top-bar-value-confirmed", id='recovered-card'),
                    html.P("Recovered", className="card-text", style={'font-size':'18px'}),
                ]
            ),
            className=f"top-bar-card-confirmed",
        )


############################################

table1 = dash_table.DataTable(df_top10.to_dict('records'), [{"name": i, "id": i} for i in df_top10.columns],
                                                    page_size=10,
                                                    
                                                    column_selectable="single",
                                                    style_as_list_view=True,
                                                    
                                                    fill_width=False,
                                                    style_table={
                                                        "width": "100%",
                                                        "height": "100vh",
                                                    },
                                                    style_header={
                                                        "backgroundColor": color_bg,
                                                        "border": color_bg,
                                                        "fontWeight": "bold",
                                                        "font": "Lato, sans-serif",
                                                        "height": "2vw",
                                                    },
                                                    style_cell={
                                                        "font-size": font_size_body,
                                                        "font-family": "Lato, sans-serif",
                                                        "border-bottom": "0.01rem solid #313841",
                                                        "backgroundColor": "#010915",
                                                        "color": "#FEFEFE",
                                                        "height": "2.75vw",
                                                    },
                                                    )

In [20]:
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div(
                        [
                            dbc.Navbar(
                                        navbar,
                                        id="navbar-content",
                                        color="#010915",
                                        dark=True,
#                                         className="desktop-navbar",
                                        
                                    ),
                            dbc.Row(
                                [
                                    dbc.Col([
                                        dbc.Row([
                                             dcc.Dropdown(id='dropdown',
                                                 options=STATE_LABELS,
                                                 value='ALL',
#                                                  className="dash-bootstrap",
                                                 style={'padding-bottom': '3%'},
#                                                  label="Secondary", color="secondary", className="m-1"
                                                ),
                                           
                                            ]),
                                        
                                        dbc.Row(
                                            [dcc.DatePickerSingle(
                                                    id='date-picker-single',
                                                    date=df.Date.max(),
                                                    
                                                )], style={'width':'300px'})
                                            ],
                                            
                                            
                                    style={'padding-left': '3%'}
                                           ),

#                                     dbc.Col(dbc.Card(confirmed_card, color="primary", inverse=True)),
                                    dbc.Col(confirmed_card, className="top-bar-card-body", width=3, 
                                        ),

                                    dbc.Col(deaths_card, className="top-bar-card-body", width=3,
                                        ),

                                    dbc.Col(recovered_card, className="top-bar-card-body", width=3,
                                        ),
                                ],
                                className="mb-4",
                                style={'padding-top': '5%'}
                            ),
#############################################################################################################################                        
                            
#                             dbc.Row([
#                                 dbc.Col([dcc.Graph(id='date_fig')]),

#                             ]),
                            
                            
                            dbc.Row([
                                dbc.Col([
                                    dbc.Container(id='table_1')
                                ]),
                                dbc.Col([dcc.Graph(id='date_fig')])
                            ], style={'padding-top': '1%'}),
                            
                            
#############################################################################################################################
           
                            
                            dbc.Row([
                                dcc.Graph(id='line_fig'),  
                            ], className="mb-4", style={'padding-left': '1%'}),
                        
                            
#############################################################################################################################
           
                            
                            dbc.
                            Row([
                                dcc.Dropdown(
                                    id='dropmenu',
                                    options=['bar' , 'scatter', 'pie'],
                                    value='bar',
                                    
                                    style={'width': '300px', 
                                          'padding-left': '3%'}
                                ),

                            ], className="mb-4",),
        
                            dbc.Row([
                                dbc.Col([
                                    dcc.Graph(
                                     id = 'my_Graph'
                                   ),
                                ], className="mb-4",),
                                
                                dbc.Col([
                                    dcc.Graph(
                                     id = 'my_Graph1',
                                        className="top-bottom-right-chart-figure"
                                   ),
                                ], className="mb-4",),
                                
                                dbc.Col([
                                    dcc.Graph(
                                     id = 'my_Graph2'
                                   ),
                                ]),
                            
                            ], className="mb-3",),
                         
                            
                    ])




@app.callback(
    Output('confirmed-card','children'),
    Output('deaths-card','children'),
    Output('recovered-card','children'),
    Output('date_fig', 'figure'),
    Output('line_fig', 'figure'),
    Output('table_1', 'children'),
    Input("dropdown", "value"),
    Input('date-picker-single', 'date')
    
)  # pylint: disable=W0612

def update_cards(country, date):
    filtered_graph = df_grouped_date.loc[date].reset_index(level=0)
    
    fig = px.scatter_geo(filtered_graph, 
                     locations="Country/Region",
                     hover_name="Country/Region",
                     hover_data=['Deaths'],
                     locationmode='country names',
                     scope='world',
                     size='Confirmed',
                     
                     template='plotly_dark',
                     size_max=30,
                     
                 )
    
    fig.update_layout(
#         title_text="Confirmed Cases",
#     autosize=True,
#         width=800,
#     height=500,
    
        title={
        'text': f"Confirmed Cases \n{date}",
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        font=dict(
        family="Courier New, monospace",
        size=18,
        
    )
        
        
    )
    
    ########################################################################
    
    ww = df_grouped_date.loc[date].reset_index(level=0)
    df_top10 = ww.sort_values(by=['Confirmed'], ascending=[False]).head(10)
    df_top10 = df_top10[['Country/Region', 'Confirmed', 'Deaths']]
    
    
    table1 = dash_table.DataTable(df_top10.to_dict('records'), [{"name": i, "id": i} for i in df_top10.columns],
                                                    page_size=10,
                                                    
                                                    column_selectable="single",
                                                    style_as_list_view=True,
                                                    
                                                    fill_width=False,
                                                    style_table={
                                                        "width": "100%",
                                                        "height": "100vh",
                                                    },
                                                    style_header={
                                                        "backgroundColor": color_bg,
                                                        "border": color_bg,
                                                        "fontWeight": "bold",
                                                        "font": "Lato, sans-serif",
                                                        "height": "2vw",
                                                    },
                                                    style_cell={
                                                        "font-size": font_size_body,
                                                        "font-family": "Lato, sans-serif",
                                                        "border-bottom": "0.01rem solid #313841",
                                                        "backgroundColor": "#010915",
                                                        "color": "#FEFEFE",
                                                        "height": "2.75vw",
                                                    },
                                                    )
    
    #######################################################################

    
    line_fig = None

    if country != 'ALL':
        
        df_line = df.groupby(by=["Country/Region", 'Date']).sum().loc[country].reset_index()
        line_fig = px.line(df_line, x='Date', y=["Confirmed", 'Deaths', 'Recovered' ], template='plotly_dark')
        
        line_fig.update_layout(
#         title_text="Confirmed Cases",
#     autosize=True,
#         width=800,
#     height=500,
    
        title={
        'text': f"Cases in {country}",
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        font=dict(
        family="Courier New, monospace",
        size=18,
        
    ))
        
        
        stats = get_stats_by_date(country, date)
    #     return stats['Confirmed'], stats['Deaths'], stats['Recovered']
        return f"{stats['Confirmed']:,.0f}", f"{stats['Deaths']:,.0f}",\
                f"{stats['Recovered']:,.0f}", fig, line_fig, table1

    
    
#     print(total_cases['Confirmed'], total_cases['Deaths'], total_cases['Recovered'])
    return f"{total_cases['Confirmed']:,.0f}", f"{total_cases['Deaths']:,.0f}",\
            f"{total_cases['Recovered']:,.0f}", fig, fig, table1


###########################################################################################################


@app.callback(
    Output(component_id= 'my_Graph' , component_property= 'figure'),
    Output(component_id= 'my_Graph1' , component_property= 'figure'),
    Output(component_id= 'my_Graph2' , component_property= 'figure'),
    Input(component_id = 'dropmenu' , component_property= 'value' )
)


def update_My_Div(selected_cont):
    
    if selected_cont == 'bar':
#         go.Figure(data=go.Bar(x=['a', 'b', 'c', 'd'], y=[1, 2, 3, 4]),
#                                layout=dict(template='plotly_dark')))
        fig_case = px.bar(df_1, x = 'Continent', y =  'Confirmed' ,  hover_data=['Continent', 'New cases'],
                           labels={'New cases':'Total of Cases of Continent'}, template='plotly_dark')
        
        fig_new_death = px.bar(df_1 , x = 'Continent' ,  y =  'Deaths' , hover_data=['Continent', 'New deaths'],
                           labels={'New deaths':'Total of Deaths of Continent'}, template='plotly_dark')
        
        fig_new_recoverd = px.bar(df_1 , x = 'Continent' , y =  'Recovered' , hover_data=['Continent', 'New recovered'],
                           labels={'New recovered':'Total of Recovered of Continent'}, template='plotly_dark')
    
    elif selected_cont == 'scatter':
        
        fig_case = px.scatter(df_1, x="Continent", y="Confirmed",
                      hover_name="Continent",
                      size = 'Confirmed' , size_max=50, template='plotly_dark')
        
        fig_new_death = px.scatter(df_1, x="Continent", y="Deaths",
                      hover_name="Continent",
                       size = 'Deaths' ,size_max=50, template='plotly_dark')
        
        fig_new_recoverd = px.scatter(df_1, x="Continent", y="Recovered",
                     size = 'Recovered', hover_name="Continent",
                      size_max=50, template='plotly_dark')

              
        
    
    else:
        
        fig_case = px.pie(df_1, values='Confirmed', names='Continent', title='Total of Confirmed of Continent' , hole=.3, template='plotly_dark')
        
        fig_new_death =  px.pie(df_1, values='Deaths', names='Continent', title='Total of Deaths of Continent' , hole=.3, template='plotly_dark')
        
        fig_new_recoverd =  px.pie(df_1, values='Recovered', names='Continent', title='Total of Recovered of Continent' , hole=.3, template='plotly_dark')
        
        

    return fig_case , fig_new_death , fig_new_recoverd 
 

app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2022 16:33:50] "GET /_alive_4afe12d6-f068-46d3-bb15-f5544e38d8a9 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [26/May/2022 16:33:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 16:33:55] "GET /assets/style.css?m=1651132982.72639 HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 16:33:55] "GET /_dash-component-suites/dash/deps/react@16.v2_4_1m1652709166.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 16:33:55] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_4_1m1652709166.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 16:33:55] "GET /assets/css/mobile.css?m=1587359534.0 HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 16:33:55] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_4_1m1652709166.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 16:33:55] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_4_1m1652709166.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 16:33:55] "GET /assets/css/common.css?m=1587359534.0 HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 16:33:55] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_